https://huggingface.co/jxm/cde-small-v1


In [1]:
import transformers

tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")

/Users/maxwoolf/Library/Mobile Documents/com~apple~CloudDocs/PythonProjects/imdb-embeddings/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [29]:
device = "mps"

model = transformers.AutoModel.from_pretrained(
    "jxm/cde-small-v1", trust_remote_code=True
)

_ = model.to(device)

/Users/maxwoolf/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/c1b1fd7a715b8eb2e232d34593154ac782c98ac9/modeling_hf_nomic_bert.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related

Disabled 37 dropout modules from model type <class 'transformers_modules.jxm.cde-small-v1.10f081914816933d06b18c414c3484c96de49cdb.model.BiEncoder'>
modified 12 rotary modules – set rotary_start_pos to 512
Disabled 74 dropout modules from model type <class 'transformers_modules.jxm.cde-small-v1.10f081914816933d06b18c414c3484c96de49cdb.model.DatasetTransformer'>


In [3]:
model.device

device(type='cpu')

In [7]:
import polars as pl
import os

In [49]:
data_dir = "/Users/maxwoolf/Downloads"

df = pl.read_parquet(os.path.join(data_dir, "test_movie_json_input.parquet"))
df = df.sample(fraction=1.0, shuffle=True, seed=42)
df

tconst,startYear,averageRating,json
str,i64,f64,str
"""tt0136535""",1998,4.8,"""{'title': 'Webmaster', 'genres…"
"""tt1933566""",1968,4.6,"""{'title': 'The daughter of the…"
"""tt2294804""",2008,7.3,"""{'title': 'Found Footage Festi…"
"""tt5297478""",2015,7.5,"""{'title': 'Boudewijn de Groot …"
"""tt0072358""",1974,6.5,"""{'title': 'Last Moments', 'gen…"
…,…,…,…
"""tt1241016""",2008,8.0,"""{'title': 'Family', 'genres': …"
"""tt19864536""",2022,6.9,"""{'title': 'Afrika', 'genres': …"
"""tt0810444""",2006,3.3,"""{'title': 'The Pumpkin Karver'…"


## First Stage


In [41]:
import torch
from tqdm.autonotebook import tqdm
import random

In [50]:
random.seed(42)

json_docs = df["json"].to_list()[:1600]

json_docs[:5]

["{'title': 'Webmaster', 'genres': ['Sci-Fi', 'Thriller'], 'is_adult': False, 'release_year': 1998, 'runtime_minutes': 102, 'directors': ['Thomas Borch Nielsen'], 'writers': ['Thomas Borch Nielsen'], 'producers': ['Vincent J. Francillon', 'Søren Juul Petersen'], 'actors': ['Lars Bom', 'Puk Scharbau', 'Jørgen Kiil', 'Karin Rørbeck', 'Mads Parsum', 'Dorte Westh Lehrmann', 'Flemming Jetmar', 'Sonnich Morgenstern', 'Kim Jansson', 'Dennis Dupont'], 'principals': [{'Lars Beyer': 'cinematographer'}, {'Kasper Leick': 'editor'}, {'Emilie Brandt Luke': 'casting_director'}, {'Thomas Ravn': 'production_designer'}]}",
 "{'title': 'The daughter of the king of fairies', 'genres': ['Drama', 'Romance'], 'is_adult': False, 'release_year': 1968, 'runtime_minutes': 104, 'directors': ['Shapur Gharib'], 'writers': ['Shapur Gharib'], 'producers': ['Abbas Homayoun', 'Iraj Sadeghpour', 'Manoochehr Vossoogh'], 'actors': ['Fourouzan', 'Manoochehr Vossoogh', 'Iran Ghaderi', 'Ahmad Ghadakchian', 'Ebrahim Fakhar', 

In [51]:
document_prefix = "search_document: "

minicorpus_size = model.config.transductive_corpus_size
minicorpus_docs = random.sample(json_docs, minicorpus_size)
len(minicorpus_docs)

512

In [52]:
minicorpus_tokens = tokenizer(
    [document_prefix + doc for doc in minicorpus_docs],
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors="pt",
).to(device)

minicorpus_tokens["input_ids"].size()

torch.Size([512, 386])

In [53]:
batch_size = 16

dataset_embeddings = []
for i in tqdm(range(0, len(minicorpus_tokens["input_ids"]), batch_size)):
    minicorpus_docs_batch = {
        k: v[i : i + batch_size] for k, v in minicorpus_tokens.items()
    }
    with torch.no_grad():
        dataset_embeddings.append(model.first_stage_model(**minicorpus_docs_batch))

dataset_embeddings = torch.cat(dataset_embeddings)
dataset_embeddings.size()

  0%|          | 0/32 [00:00<?, ?it/s]

torch.Size([512, 768])

In [54]:
import torch
from tqdm.autonotebook import tqdm

docs = tokenizer(
    [document_prefix + doc for doc in json_docs],
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors="pt",
).to(device)

doc_embeddings = []
for i in tqdm(range(0, len(docs["input_ids"]), batch_size)):
    docs_batch = {k: v[i : i + batch_size] for k, v in docs.items()}
    with torch.no_grad():
        doc_embeddings_batch = model.second_stage_modesl(
            input_ids=docs_batch["input_ids"],
            attention_mask=docs_batch["attention_mask"],
            dataset_embeddings=dataset_embeddings,
        )
        doc_embeddings.append(doc_embeddings_batch)

  0%|          | 0/100 [00:00<?, ?it/s]

TypeError: cat(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor

In [55]:
doc_embeddings = torch.cat(doc_embeddings)
doc_embeddings /= doc_embeddings.norm(p=2, dim=1, keepdim=True)
doc_embeddings.size()

torch.Size([1600, 768])